In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 3, Finished, Available, Finished)

In [2]:
DeltaTable.createIfNotExists(spark)\
          .tableName("Dim_OrderPriority")\
          .addColumn("OrderPriority_ID",LongType())\
          .addColumn("Order_Priority",StringType())\
          .addColumn("Created_TS",TimestampType())\
          .addColumn("Modified_TS",TimestampType())\
          .execute()

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 4, Finished, Available, Finished)

In [3]:
df = spark.read.table("Lakehouse_Demo.Dim_OrderPriority")

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 5, Finished, Available, Finished)

In [5]:
Max_Date = df.selectExpr("coalesce(max(Modified_TS),'1900-01-01')").first()[0]
print(Max_Date)

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 7, Finished, Available, Finished)

1900-01-01


In [9]:
df_Sales = spark.read.table("Lakehouse_Demo.Sales_Stg")
display(df_Sales.limit(5))

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 72f23d76-0e03-49d3-874d-4c038f7c0532)

In [10]:
df_Sales_mod = df_Sales.select("Order_Priority").where(col("Modified_TS")>Max_Date).drop_duplicates()
display(df_Sales_mod.limit(5))

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 78fbbba7-f660-4ea1-b27a-a0f7b44db141)

In [13]:
Max_ID = df.selectExpr("coalesce(max(OrderPriority_ID),0)").first()[0]
print(Max_ID)


StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 15, Finished, Available, Finished)

0


In [14]:
df_final = df_Sales_mod.withColumn("OrderPriority_ID",Max_ID+monotonically_increasing_id()+1)

display(df_final.limit(10))

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a3f9e405-1b83-4d6b-b384-d048274fbc95)

In [18]:
df_OrderPriority = DeltaTable.forName(spark, "Lakehouse_Demo.Dim_OrderPriority")

df_stg = df_final

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 20, Finished, Available, Finished)

In [19]:
from pyspark.sql.functions import current_timestamp

df_OrderPriority.alias("op") \
    .merge(
        df_stg.alias("stg"),
        "op.Order_Priority = stg.Order_Priority"
    ) \
    .whenMatchedUpdate(
        set={
            "Modified_TS": current_timestamp()
        }
    ) \
    .whenNotMatchedInsert(
        values={
            "OrderPriority_ID": "stg.OrderPriority_ID",
            "Order_Priority": "stg.Order_Priority",
            "Created_TS": current_timestamp(),
            "Modified_TS": current_timestamp()
        }
    ) \
    .execute()


StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 21, Finished, Available, Finished)

In [20]:
%%sql
select * from Lakehouse_Demo.Dim_OrderPriority

StatementMeta(, 77bdb1e3-24cf-42f5-800c-b4fdbd02703f, 22, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>